In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pickle
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np

#wirite ML-model to model_pickle file
#with open('model_pickle', 'wb') as f:
#    pickle.dump(model, f)

#save DL-model with keras.models.save_model
#tf.keras.models.save_model(model, export_path,overwrite=True,include_optimizer=True,
#    save_format=None, signatures=None,    options=None)

#MODEL_DIR='/content/drive/My Drive/master_project/CNN_Model.h5' #todo: get it from database?



In [ ]:
import enum
# Using enum class create enumerations
class Data_type(enum.Enum):
   ECG=0
   BLOODCELL=1

class ML_type(enum.Enum):
  DL=0
  ML=1

In [ ]:
class Modeling():
  
  def __init__(self,data_type,ML_type):
    self.data_type=data_type
    self.ML_type=ML_type
    #print(self.data_type)
    #print(self.ML_type)
    self.model=self.load_model()
    

  def load_model(self):    
      MODEL_DIR=self.get_model_path() #store in database?
      #print(MODEL_DIR)
      if self.ML_type==ML_type.DL:
      # loading DL model 
        return load_model(MODEL_DIR) 
      #read Machine Learning model from model_pickle file
      if ML_type==ML_type.ML:
        with open(MODEL_DIR, 'rb') as f:
          return pickle.load(f)    

  def get_model_path(self):
    #todo: get it from database?
    if self.data_type==Data_type.ECG:
      return '/content/drive/My Drive/master_project/ECG_Model.h5' 
    if self.data_type==Data_type.BLOODCELL:
      return '/content/drive/My Drive/master_project/Blood_Cell_Model.h5'

  def model_predict(self,df):
    """
    to predict the input df

    """
    return self.model.predict(df)

In [ ]:
model=Modeling(Data_type.ECG,ML_type.DL)

Data_type.ECG
ML_type.DL
/content/drive/My Drive/master_project/ECG_Model.h5


In [ ]:
model.model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 30000, 4)          1264      
_________________________________________________________________
average_pooling1d_9 (Average (None, 7500, 4)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 7500, 16)          1488      
_________________________________________________________________
average_pooling1d_10 (Averag (None, 1875, 16)          0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 1875, 32)          12832     
_________________________________________________________________
average_pooling1d_11 (Averag (None, 235, 32)           0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 235, 64)          

In [ ]:
symbol_names=['i','ii','iii','avr','avl','avf','v1','v2','v3','v4','v5','v6','vx','vy','vz']
data_path='/content/drive/My Drive/physionet.org/files/ptbdb/'
def read_data_json(f,symbol_names): 
  df_read=pd.read_json(f)
  for symbol in symbol_names:
    df_read[symbol]=df_read[symbol].apply(np.array)
  return df_read

In [ ]:
symbol_names=['i','ii','iii','avr','avl','avf','v1','v2','v3','v4','v5','v6','vx','vy','vz']
data_path='/content/drive/My Drive/physionet.org/files/ptbdb/'
def str_to_array(str_data):
  num_list=[]
  str_list=str_data.split(" ")
  for i in range(len(str_list)-1):
    num_list.append(float(str_list[i]))  
  return np.array(num_list)

def read_data_csv(f,symbol_names): 
  df_read=pd.read_csv(f,index_col=0)
  for symbol in symbol_names:
    df_read[symbol]=df_read[symbol].apply(str_to_array)
  return df_read

In [ ]:
%cd /content/drive/My Drive/physionet.org/files/ptbdb/

/content/drive/My Drive/physionet.org/files/ptbdb


In [ ]:
!ls

1.0.0		   1.0.0data_all.csv   1.0.0data_group3.json  data_all.json
1.0.0data_41.json  1.0.0data_all.json  data_41.json	      data_group3.csv


In [ ]:
data_read_from_csv=read_data_csv(data_path+'data_group3.csv',symbol_names)

In [ ]:
clean_data=data_read_from_csv[symbol_names].reindex()

In [ ]:
clean_data.head()

,i,ii,iii,avr,avl,avf,v1,v2,v3,v4,v5,v6,vx,vy,vz
0,"[-0.24450000000000002, -0.24250000000000002, -...","[-0.229, -0.2335, -0.23450000000000001, -0.229...","[0.0155, 0.009000000000000001, 0.007, 0.012, 0...","[0.23700000000000002, 0.23800000000000002, 0.2...","[-0.13, -0.1255, -0.124, -0.126, -0.1180000000...","[-0.107, -0.1125, -0.114, -0.109, -0.1115, -0....","[-0.044, -0.042, -0.044, -0.045, -0.0455, -0.0...","[-0.12050000000000001, -0.11750000000000001, -...","[-0.056, -0.051000000000000004, -0.05350000000...","[0.106, 0.1095, 0.1095, 0.1115, 0.1115, 0.1075...","[0.1965, 0.202, 0.2005, 0.201, 0.2, 0.196, 0.1...","[0.195, 0.198, 0.1965, 0.197, 0.1985, 0.194, 0...","[-0.0015, -0.0015, -0.0035, -0.002, 0.0005, 0....","[0.06, 0.061, 0.0555, 0.053500000000000006, 0....","[-0.009000000000000001, -0.01, -0.0085, -0.008..."
1,"[-0.47400000000000003, -0.444, -0.4555, -0.462...","[-0.059500000000000004, -0.032, -0.0345, -0.03...","[0.41500000000000004, 0.41250000000000003, 0.4...","[0.267, 0.23800000000000002, 0.245500000000000...","[-0.445, -0.42850000000000005, -0.439, -0.4475...","[0.178, 0.191, 0.194, 0.2005, 0.212, 0.2135000...","[1.252, 1.2575, 1.256, 1.256, 1.2605, 1.2545, ...","[1.287, 1.2845, 1.282, 1.2775, 1.276, 1.2705, ...","[-0.9325, -0.929, -0.936, -0.9420000000000001,...","[-0.8220000000000001, -0.8215, -0.824000000000...","[1.6139999999999999, 1.6195, 1.6195, 1.6185, 1...","[1.708, 1.712, 1.7125, 1.7125, 1.7165, 1.716, ...","[0.5365, 0.5335, 0.5305, 0.531, 0.5305, 0.53, ...","[-1.1385, -1.1385, -1.141, -1.133, -1.1265, -1...","[0.9675, 0.9645, 0.961, 0.963, 0.962, 0.9595, ..."
2,"[0.254, 0.263, 0.2755, 0.28, 0.3035, 0.305, 0....","[0.7535000000000001, 0.7615000000000001, 0.786...","[0.499, 0.498, 0.51, 0.5245000000000001, 0.537...","[-0.5035000000000001, -0.5115000000000001, -0....","[-0.12250000000000001, -0.11750000000000001, -...","[0.626, 0.6295000000000001, 0.648, 0.6645, 0.6...","[0.5815, 0.5955, 0.6015, 0.6015, 0.6105, 0.613...","[0.7875000000000001, 0.8015, 0.8075, 0.8115, 0...","[0.936, 0.9535, 0.9570000000000001, 0.962, 0.9...","[0.984, 1.001, 1.0065, 1.0125, 1.0235, 1.024, ...","[0.8180000000000001, 0.8335, 0.833, 0.8385, 0....","[0.5385, 0.551, 0.5515, 0.5605, 0.581, 0.5755,...","[0.385, 0.38, 0.382, 0.382, 0.384, 0.3825, 0.3...","[-0.131, -0.132, -0.11800000000000001, -0.1075...","[0.029, 0.024, 0.0245, 0.0275, 0.028, 0.0205, ..."
316,"[0.031, 0.029, 0.03, 0.0275, 0.029500000000000...","[-1.1485, -1.1485, -1.145, -1.1475, -1.149, -1...","[-1.1795, -1.1775, -1.175, -1.175, -1.1785, -1...","[0.5585, 0.5595, 0.557, 0.5595, 0.5595, 0.5585...","[0.6055, 0.6035, 0.603, 0.6015, 0.6045, 0.6085...","[-1.164, -1.163, -1.1595, -1.161, -1.1635, -1....","[0.166, 0.1675, 0.1655, 0.162, 0.162, 0.1655, ...","[0.258, 0.2585, 0.2585, 0.259, 0.2575, 0.2615,...","[0.5730000000000001, 0.5735, 0.576000000000000...","[0.129, 0.128, 0.129, 0.128, 0.1275, 0.1305, 0...","[-0.3825, -0.3835, -0.381, -0.382, -0.382, -0....","[0.012, 0.014, 0.015, 0.016, 0.0155, 0.0195, 0...","[-0.3415, -0.338, -0.337, -0.336, -0.3355, -0....","[-0.6980000000000001, -0.6985, -0.696000000000...","[0.0095, 0.0095, 0.006500000000000001, 0.00750..."
317,"[-0.658, -0.66, -0.6575000000000001, -0.652, -...","[-0.1105, -0.1085, -0.10300000000000001, -0.10...","[0.5475, 0.5515, 0.5545, 0.547, 0.531, 0.51850...","[0.3845, 0.3845, 0.3805, 0.379, 0.378, 0.379, ...","[-0.6025, -0.606, -0.606, -0.5995, -0.587, -0....","[0.219, 0.222, 0.226, 0.221, 0.209500000000000...","[0.5235000000000001, 0.5305, 0.536, 0.545, 0.5...","[0.649, 0.669, 0.6900000000000001, 0.7165, 0.7...","[0.1675, 0.201, 0.231, 0.266, 0.3005, 0.328, 0...","[0.07, 0.0995, 0.129, 0.1585, 0.1885, 0.210500...","[-0.1915, -0.1665, -0.14300000000000002, -0.11...","[0.445, 0.4595, 0.47800000000000004, 0.4955, 0...","[-0.07250000000000001, -0.060500000000000005, ...","[1.6095000000000002, 1.629, 1.6465, 1.661, 1.6...","[0.5455, 0.5215000000000001, 0.503, 0.491, 0.4..."


In [ ]:
#create a new shape of the data
temp=[]
for i in clean_data.index:    
    for k in range(30000):
      for name in symbol_names:
        temp.append(clean_data[name][i][k])
frames=np.array(temp).reshape(clean_data.shape[0],30000,len(symbol_names))

In [ ]:
preds=model.model_predict(frames)

In [ ]:
np.argmax(preds[0])

3